In [1]:
import sys

sys.path.append("..")

from itertools import product
import pandas as pd
import kornia.augmentation as K
import torch
import torch.nn as nn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from src.models import get_model_by_name
from src.datasets.resisc45 import RESISC45Minimal
from src.utils import extract_features
from tqdm import tqdm

device = torch.device("cuda:1")

In [2]:
# Fit and save features
model_names = [
    "resnet50_pretrained_imagenet",
    "resnet50_pretrained_seco",
    "resnet50_pretrained_moco",
    "resnet50_randominit",
    "imagestats",
    "mosaiks_512_3",
    "mosaiks_zca_512_3",
]
sizes = [256]
normalization_methods = ["divide", "standardization"]
rgb = True

columns = {
    "model_name": [],
    "rgb": [],
    "size": [],
    "normalization": [],
    "accuracy": [],
    "accuracy_scaler": [],
}

for model_name, size, normalization in tqdm(product(model_names, sizes, normalization_methods), total=7*2):
    transforms = nn.Sequential(K.Resize(size)).to(device)

    dm = RESISC45Minimal(
        root="../data/resisc45//",
        normalization_method=normalization,
        batch_size=64,
        num_workers=8
    )
    dm.setup()

    model = get_model_by_name(model_name, rgb, device=device, dataset=dm.train_dataset)

    x_train, y_train = extract_features(
        model, dm.train_dataloader(), device, transforms=transforms, verbose=False
    )
    x_test, y_test = extract_features(
        model, dm.test_dataloader(), device, transforms=transforms, verbose=False
    )

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(x_train, y_train)
    acc = knn_model.score(x_test, y_test)

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(x_train, y_train)
    acc_scaler = knn_model.score(x_test, y_test)

    columns["model_name"].append(model_name)
    columns["rgb"].append(rgb)
    columns["size"].append(size)
    columns["normalization"].append(normalization)
    columns["accuracy"].append(acc)
    columns["accuracy_scaler"].append(acc_scaler)

100%|██████████| 14/14 [10:39<00:00, 45.68s/it]


In [3]:
df = pd.DataFrame(columns)

In [4]:
df.to_csv("../results/resisc45_table_results.csv")

In [5]:
df

,model_name,rgb,size,normalization,accuracy,accuracy_scaler
0,resnet50_pretrained_imagenet,True,256,divide,0.749524,0.658254
1,resnet50_pretrained_imagenet,True,256,standardization,0.774762,0.695556
2,resnet50_pretrained_seco,True,256,divide,0.496825,0.576349
3,resnet50_pretrained_seco,True,256,standardization,0.159365,0.200317
4,resnet50_pretrained_moco,True,256,divide,0.713333,0.732381
5,resnet50_pretrained_moco,True,256,standardization,0.364603,0.393016
6,resnet50_randominit,True,256,divide,0.280635,0.304762
7,resnet50_randominit,True,256,standardization,0.347302,0.384127
8,imagestats,True,256,divide,0.340159,0.340317
9,imagestats,True,256,standardization,0.338730,0.340317
